本文旨在基于百度向量数据库实现一个简单的RAG（Retrieval-Augmented Generation）示例。

## RAG介绍
RAG是一种先进的自然语言处理方法，它结合了信息检索和文本生成技术，用于提高问答系统、聊天机器人等应用的性能。以下是RAG的详细工作流程：

### RAG的工作流程

1. **文档加载（Document Loading）**
   - 从各种来源加载大量文档数据。
   - 这些文档将作为知识库，用于后续的信息检索。

2. **文档分割（Document Splitting）**
   - 将加载的文档分割成更小的段落或部分。
   - 这有助于提高检索的准确性和效率。

3. **嵌入向量生成（Embedding Generation）**
   - 对每个文档或文档的部分生成嵌入向量。
   - 这些嵌入向量捕捉文档的语义信息，方便后续的相似度比较。

4. **写入向量数据库（Writing to Vector Database）**
   - 将生成的嵌入向量存储在一个向量数据库中。
   - 数据库支持高效的相似度搜索操作。

5. **查询生成（Query Generation）**
   - 用户提出一个问题或输入一个提示。
   - RAG模型根据输入生成一个或多个相关的查询。

6. **文档检索（Document Retrieval）**
   - 使用生成的查询在向量数据库中检索相关文档。
   - 选择与查询最相关的文档作为信息源。

7. **上下文融合（Context Integration）**
   - 将检索到的文档内容与原始问题或提示融合，构成扩展的上下文。

8. **答案生成（Answer Generation）**
   - 基于融合后的上下文，RAG生成模型产生最终的回答或文本。
   - 这一步骤旨在综合原始输入和检索到的信息。

## 文档加载（Document Loading）

In [ ]:
!pip install langchain
!pip install pymochow
!pip install qianfan
!pip install pdfplumber

In [4]:
from langchain_community.document_loaders import PDFPlumberLoader
loader = PDFPlumberLoader("./example_data/ai-paper.pdf")
documents = loader.load()
documents[0]

Document(page_content='Feature Article | 特稿\n大模型发展趋势及国内外研究现状\n◎ 撰文 | 熊子晗 李雨轩 陈军 陈大北\n大模型是“大算力+强算法”结合的产物,通常是在大规模无标注数据上进行训练,学习出一种特征\n和规则。基于大模型进行应用开发时,将大模型进行微调,如在下游特定任务上的小规模有标注数据上进\n行二次训练,或者不进行微调,就可以完成多个应用场景的任务。\n业、地球科学、航空航天等领域开始从偏微分方程\n大模型发展趋势 的方法拓展到AI方法。\n国际数据公司（IDC）认为，大模型的发展是\n华为认为，首先，大模型的出现和繁荣既是 大势所趋。首先，未来大小模型会协同进化，推动\n当前深度学习的顶峰，也代表着深度学习算法的瓶 端侧化发展。大模型负责向小模型输出模型能力，\n颈。对大模型的需求本质上是对大数据的需求。当 小模型更精确地处理自己“擅长”的任务，再将应\n前的人工智能算法尚无法高效地建模不同数据之间 用中的数据与结果反哺给大模型，让大模型持续迭\n的关系，并以此解决模型泛化的问题，取而代之， 代更新，形成大小模型协同应用模式，达到降低能\n通过收集并处理大量训练数据，人工智能算法能够 耗、提高整体模型精度的效果。其次，大模型通用\n通过“死记硬背”的方式一定程度上提升泛化能 性持续增强，实现AI开发“大一统”模式。大模\n力。从这一角度看，大模型对数据的应用依然处于 型由于其泛化性、通用性为人工智能带来了新机\n比较初级、低效的水平。可以预见，这种方式的边 遇。目前，在通用模型的基础上，各行业正利用精\n际效应明显，数据集越大模型越大，提升同等精度 调或提示语prompt的方式加入任务间的差异化\n所需要的代价就越大。要想通过预训练大模型真正 内容，从而极大地提高了模型的利用率，推动AI\n解决人工智能问题，看来不太现实。其次，除了在 开发走向“统一”。最后，大模型从科研创新走向\n数据集构建、模型设计乃至评测标准方面持续演 产业落地，通过开放的生态持续释放红利。大模型\n进，业界首先需要做的是抛弃预训练大模型“参数 最重要的优势是推动AI 进入大规模可复制的产业\n量至上”的评判标准。因此，参数量并不是评判模 落地阶段，仅需零样本、小样本的学习就可以达到\n型能力的最好标准——如何用好参数并将模型的

## 文档分割（Document Splitting）

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 384, chunk_overlap = 0, separators=["\n\n", "\n", " ", "", "。", "，"])
all_splits = text_splitter.split_documents(documents)
all_splits[0]

Document(page_content='Feature Article | 特稿\n大模型发展趋势及国内外研究现状\n◎ 撰文 | 熊子晗 李雨轩 陈军 陈大北\n大模型是“大算力+强算法”结合的产物,通常是在大规模无标注数据上进行训练,学习出一种特征\n和规则。基于大模型进行应用开发时,将大模型进行微调,如在下游特定任务上的小规模有标注数据上进\n行二次训练,或者不进行微调,就可以完成多个应用场景的任务。\n业、地球科学、航空航天等领域开始从偏微分方程\n大模型发展趋势 的方法拓展到AI方法。\n国际数据公司（IDC）认为，大模型的发展是\n华为认为，首先，大模型的出现和繁荣既是 大势所趋。首先，未来大小模型会协同进化，推动\n当前深度学习的顶峰，也代表着深度学习算法的瓶 端侧化发展。大模型负责向小模型输出模型能力，\n颈。对大模型的需求本质上是对大数据的需求。当 小模型更精确地处理自己“擅长”的任务，再将应', metadata={'source': './example_data/ai-paper.pdf', 'file_path': './example_data/ai-paper.pdf', 'page': 0, 'total_pages': 7, 'Producer': 'TTKN', 'CreationDate': "D:20230718144804-08'00'", 'Author': 'CNKI', 'Creator': 'ReaderEx_DIS 2.3.0 Build 4031'})

## 嵌入向量生成（Embedding Generation）
这里使用了百度千帆平台进行向量生产，具体可以参考[千帆的使用文档](https://cloud.baidu.com/doc/WENXINWORKSHOP/s/hlmokk9qn)

In [49]:
import os
import qianfan
import time

# 使用安全认证AK/SK鉴权，通过环境变量方式初始化；替换下列示例中参数，安全认证Access Key替换your_iam_ak，Secret Key替换your_iam_sk
os.environ["QIANFAN_ACCESS_KEY"] = "your_ak"
os.environ["QIANFAN_SECRET_KEY"] = "your_sk"

emb = qianfan.Embedding()

embeddings = []
for chunk in all_splits:
    resp = emb.do(texts=[  # 省略 model 时则调用默认模型 Embedding-V1
        chunk.page_content
    ])
    embeddings.append(resp['data'][0]['embedding'])
    time.sleep(1)
embeddings[0]

[INFO] [03-07 00:02:09] openapi_requestor.py:316 [t:8618713344]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-07 00:02:10] openapi_requestor.py:316 [t:8618713344]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-07 00:02:11] openapi_requestor.py:316 [t:8618713344]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-07 00:02:12] openapi_requestor.py:316 [t:8618713344]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-07 00:02:13] openapi_requestor.py:316 [t:8618713344]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-07 00:02:14] openapi_requestor.py:316 [t:8618713344]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-07 00:02:16] openapi_requestor.py:316 [t:8618713344]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-07 00:02:17] openapi_requestor.py:316 [t:8618713344]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [03-07 00:02:18] openapi_requestor.py:316

[0.11224345862865448,
 0.054152220487594604,
 -0.00048503236030228436,
 -0.02061440609395504,
 -0.03978653624653816,
 -0.13321490585803986,
 0.019213810563087463,
 -0.08260218054056168,
 -0.07853612303733826,
 0.0272421445697546,
 0.012822103686630726,
 0.007456572260707617,
 0.021469425410032272,
 -0.046562984585762024,
 -0.16177448630332947,
 -0.02001659944653511,
 0.018230443820357323,
 -0.0613895058631897,
 -0.09145896136760712,
 0.04688601940870285,
 0.0007361111929640174,
 0.00711675314232707,
 -0.06621534377336502,
 -0.05942794308066368,
 -0.05141240730881691,
 0.04894018545746803,
 0.010030743665993214,
 -0.03037424385547638,
 0.1294194459915161,
 0.03385620191693306,
 -0.06916307657957077,
 -0.10393933951854706,
 0.025252867490053177,
 -0.08275408297777176,
 0.029970388859510422,
 0.030179088935256004,
 0.06368114054203033,
 -0.00975083839148283,
 0.04710584133863449,
 -0.031163427978754044,
 -0.00028807061607949436,
 -0.11335715651512146,
 -0.03873395919799805,
 -0.0303310658

## 写入向量数据库（Writing to Vector Database)

将基于原始文档生产的标量和向量数据写入到向量数据库主要分为以下三步：
1. ** 购买[百度向量数据库实例](https://cloud.baidu.com/doc/VDB/s/hlrsoazuf) **
2. ** 创建数据库、数据表 **
3. ** 写入数据 **

### 创建数据库、数据表

In [ ]:
import pymochow
from pymochow.configuration import Configuration

account = 'root'
api_key = 'your api key'
endpoint = 'your endpoint' #example:http://127.0.0.1:8511

config = Configuration(credentials=BceCredentials(account, api_key), endpoint=endpoint)
client = pymochow.MochowClient(config)

In [ ]:
db=client.create_database("document")
database_list = client.list_databases()
for db_item in database_list:
    print("database: {}".format(db_item.database_name))

In [ ]:
from pymochow.model.schema import Schema, Field, VectorIndex, SecondaryIndex, HNSWParams
from pymochow.model.enum import FieldType, IndexType, MetricType
from pymochow.model.table import Partition

fields = []
fields.append(
    Field(
        "id", 
        FieldType.UINT64, 
        primary_key=True,
        partition_key=True, 
        auto_increment=False, 
        not_null=True
    )
)
fields.append(Field("text", FieldType.STRING))
fields.append(Field("metadata", FieldType.STRING))
fields.append(Field("source", FieldType.STRING))
fields.append(Field("author", FieldType.STRING, not_null=True))
fields.append(
    Field(
        vector, 
        FieldType.FLOAT_VECTOR,
        len(embedding),
        not_null=True
    )
)

indexes = []
indexes.append(
    VectorIndex(
        index_name="vector_idx",
        index_type=IndexType.HNSW,
        metric_type=MetricType.L2,
        params=HNSWParams(m=32, efconstruction=200)
    )
)
indexes.append(SecondaryIndex(index_name="author_idx", field="author"))

table = db.create_table(
    table_name="chunks",
    replication=3,
    partition=Partition(partition_num=1),
    schema=Schema(fields=fields, indexes=indexes)
)


### 写入数据

In [ ]:
from pymochow.model.table import Row
import json
rows = []
for index, chunk in enumerate(all_splits):
    metadata = "{}"
    if chunk.metadata is not None:
        metadata = json.dumps(chunk.metadata)
    row = Row(
        id=index,
        text=chunk.page_content,
        metadata=metadata,
        source=chunk.metadata["source"],
        author=chunk.metadata["Creator"],
        vector=embeddings[index]
    )
    rows.append(row)
rows[0].to_dict()
table.upsert(rows=rows)

### 构建向量索引

In [ ]:
table.rebuild_index("vector_idx")

## RAG 问答示例

In [ ]:
from langchain_community.vectorstores import BaiduVectorDB
from langchain_community.vectorstores.baiduvectordb import ConnectionParams, TableParams
from langchain_community.embeddings import QianfanEmbeddingsEndpoint
from langchain_community.chat_models import QianfanChatEndpoint
from langchain.chains import RetrievalQA

# 初始化向量嵌入和连接参数
embeddings = QianfanEmbeddingsEndpoint()
conn_params = ConnectionParams(
    endpoint=config.endpoint,
    account=config.account,
    api_key=config.api_key
)

# 初始化百度云向量数据库
vector_db = BaiduVectorDB(
    embedding=embeddings,
    connection_params=conn_params,
    table_params=TableParams(384),
    database_name="document",
    table_name="chunks",
    drop_old=False,
)

# 初始化检索器和对话模型
retriever = vector_db.as_retriever(search_type="similarity")
qianfan_chat_model = QianfanChatEndpoint(model="ERNIE-Bot", temperature=0.1)

# 初始化问答模块
qa = RetrievalQA.from_chain_type(llm=qianfan_chat_model, chain_type="refine", retriever=retriever, return_source_documents=True)

# 接收用户输入的问题
query = input("\nYour question: ")

# 处理用户问题并获取答案和相关文档
res = qa(query)
answer, docs = res['result'], res['source_documents']
